In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, desc, min, max, avg, sum
import pandas as pd

In [2]:
spark = SparkSession.builder \
                    .appName("IMDb Insights with PySpark") \
                    .getOrCreate()

movies_path = "/Users/rahulsaranm/Desktop/Projects/Data Projects/IMDb Data Files/title.basics.tsv.gz"
ratings_path = "/Users/rahulsaranm/Desktop/Projects/Data Projects/IMDb Data Files/title.ratings.tsv.gz"

movies_df = spark.read.format("csv") \
                    .option("sep", "\t") \
                    .option("header", "true") \
                    .option("inferSchema", "true") \
                    .load(movies_path)

ratings_df = spark.read.format("csv") \
                    .option("sep", "\t") \
                    .option("header", "true") \
                    .option("inferSchema", "true") \
                    .load(ratings_path)

movies_df.show(5)
ratings_df.show(5)

25/02/12 18:30:12 WARN Utils: Your hostname, Rahuls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.171 instead (on interface en0)
25/02/12 18:30:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 18:30:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|              

In [3]:
movies_df.printSchema()
ratings_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [4]:
null_counts = movies_df.select(
    [sum(col(column).isNull().cast("int")).alias(column) for column in movies_df.columns]
)

# Show the result
null_counts.show()

[Stage 6:>                                                          (0 + 1) / 1]

+------+---------+------------+-------------+-------+---------+-------+--------------+------+
|tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+------+---------+------------+-------------+-------+---------+-------+--------------+------+
|     0|        0|           0|            0|      0|        0|      0|             0|   822|
+------+---------+------------+-------------+-------+---------+-------+--------------+------+



In [5]:
null_counts = ratings_df.select(
    [sum(col(column).isNull().cast("int")).alias(column) for column in ratings_df.columns]
)

# Show the result
null_counts.show()

+------+-------------+--------+
|tconst|averageRating|numVotes|
+------+-------------+--------+
|     0|            0|       0|
+------+-------------+--------+



In [6]:
movies_df = movies_df.fillna({"genres": "Unknown"})

In [7]:
null_counts = movies_df.select(
    [sum(col(column).isNull().cast("int")).alias(column) for column in movies_df.columns]
)

# Show the result
null_counts.show()

[Stage 12:>                                                         (0 + 1) / 1]

+------+---------+------------+-------------+-------+---------+-------+--------------+------+
|tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+------+---------+------------+-------------+-------+---------+-------+--------------+------+
|     0|        0|           0|            0|      0|        0|      0|             0|     0|
+------+---------+------------+-------------+-------+---------+-------+--------------+------+



In [8]:
unique_movies_df = movies_df.dropDuplicates()
print(f"Total Rows: {movies_df.count()}")
print(f"Unique Rows:{unique_movies_df.count()}")

Total Rows: 11442166


25/02/12 18:30:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:30:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/12 18:31:01 WARN RowBasedKeyValueBatch: Calling spill() on

Unique Rows:11442166


In [9]:
unique_ratings_df = ratings_df.dropDuplicates()
print(f"Total Rows: {ratings_df.count()}")
print(f"Unique Rows:{unique_ratings_df.count()}")

Total Rows: 1532242


[Stage 27:>                                                         (0 + 1) / 1]

Unique Rows:1532242


In [10]:
movies_df.printSchema()
ratings_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = false)

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [11]:
movies_df.show(5)
ratings_df.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|              

In [12]:
movies_df.select("isAdult","startYear","endYear","runtimeMinutes").show(10)

+-------+---------+-------+--------------+
|isAdult|startYear|endYear|runtimeMinutes|
+-------+---------+-------+--------------+
|      0|     1894|     \N|             1|
|      0|     1892|     \N|             5|
|      0|     1892|     \N|             5|
|      0|     1892|     \N|            12|
|      0|     1893|     \N|             1|
|      0|     1894|     \N|             1|
|      0|     1894|     \N|             1|
|      0|     1894|     \N|             1|
|      0|     1894|     \N|            45|
|      0|     1895|     \N|             1|
+-------+---------+-------+--------------+
only showing top 10 rows



In [13]:
movies_df = movies_df.withColumn("endYear",
                                 when(movies_df["endYear"] == "\\N", None).otherwise(movies_df["endYear"]))

movies_df = movies_df.withColumn("isAdult", movies_df['isAdult'].cast("boolean")) \
                     .withColumn("startYear", movies_df['startYear'].cast("int")) \
                     .withColumn("endYear", movies_df['endYear'].cast("int")) \
                     .withColumn("runtimeMinutes", movies_df['runtimeMinutes'].cast("int"))

movies_df.printSchema()
movies_df.show(10)

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: boolean (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- genres: string (nullable = false)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|  false|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|  false|     1892|   NULL|             5|     Anima

In [14]:
movies_df.orderBy(desc("endYear")).show(50)

[Stage 37:>                                                         (0 + 1) / 1]

+----------+------------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|   titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+------------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt23957098|    tvSeries|           Khud Teri|           Khud Teri|  false|     2022|   2030|          NULL|             Romance|
|tt33310042|tvMiniSeries|French Alps 2030:...|French Alps 2030:...|  false|     2030|   2030|          NULL|               Sport|
|tt30761334|tvMiniSeries|             OstyFun|          Les Mythos|  false|     2019|   2029|             3|              Comedy|
|tt32027577|    tvSeries|         Larry Movie|         Larry Movie|  false|     2023|   2029|          NULL|              Family|
|tt33333353|    tvSeries|         Odd Squad 2|         Odd Squad 2|  false|     2024|   20

In [15]:
movies_df.selectExpr("min(startYear)", "max(startYear)", "min(endYear)", "max(endYear)").show()

movies_df.groupBy("titleType").agg(min("startYear").alias("min_startYear"),
                                   max("startYear").alias("max_startYear"),
                                  ).show()

+--------------+--------------+------------+------------+
|min(startYear)|max(startYear)|min(endYear)|max(endYear)|
+--------------+--------------+------------+------------+
|          1874|          2031|           3|        2030|
+--------------+--------------+------------+------------+



[Stage 41:>                                                         (0 + 1) / 1]

+------------+-------------+-------------+
|   titleType|min_startYear|max_startYear|
+------------+-------------+-------------+
|    tvSeries|         1927|         2029|
|tvMiniSeries|         1928|         2030|
|     tvMovie|         1928|         2026|
|     tvPilot|         1991|         1991|
|   tvEpisode|         1928|         2030|
|       movie|         1894|         2031|
|   tvSpecial|         1931|         2026|
|       video|         1923|         2026|
|   videoGame|         1958|         2026|
|     tvShort|         1929|         2025|
|       short|         1874|         2030|
+------------+-------------+-------------+



In [16]:
movies_df.groupBy("titleType").agg(min("runtimeMinutes").alias("min_runtime"),
                                   max("runtimeMinutes").alias("max_runtime"),
                                   avg("runtimeMinutes").alias("avg_runtime")
                                  ).show()

[Stage 44:>                                                         (0 + 1) / 1]

+------------+-----------+-----------+------------------+
|   titleType|min_runtime|max_runtime|       avg_runtime|
+------------+-----------+-----------+------------------+
|    tvSeries|          1|    3692080| 81.34235420792555|
|tvMiniSeries|          1|      37440| 74.88083329286651|
|     tvMovie|          1|       3720| 71.79331491011357|
|     tvPilot|       NULL|       NULL|              NULL|
|   tvEpisode|          1|       6000|38.306058178051806|
|       movie|          1|      59460| 89.70951883833634|
|   tvSpecial|          1|      17280| 86.49027519974175|
|       video|          1|      35791| 69.86336429906812|
|   videoGame|          1|       1800|105.54875283446712|
|     tvShort|          1|        120|12.139262472885033|
|       short|          0|        360|13.091488564489234|
+------------+-----------+-----------+------------------+



In [17]:
# Intialize the filtered_df = movies_df, in this case we can have copy of the Dataframe to filter.
filtered_df = movies_df

conditions = when((col('titleType') == 'tvMovie') & (col('startYear') <= 2025), True) \
                .when((col('titleType') == 'movie') & (col('startYear') <= 2025), True) \
                .when((col('titleType') == 'video') & (col('startYear') <= 2025), True) \
                .when((col('titleType') == 'videoGame') & (col('startYear') <= 2025), True) \
                .when((col('titleType') == 'tvShort') & (col('startYear') <= 2025), True) \
                .when((col('titleType') == 'short') & (col('startYear') <= 2025), True) \
                .when((col('titleType') == 'tvSeries') & (col('startYear') <= 2030), True) \
                .when((col('titleType') == 'tvMiniSeries') & (col('startYear') <= 2030), True) \
                .when((col('titleType') == 'tvEpisode') & (col('startYear') <= 2030), True) \
                .when((col('titleType') == 'tvSpecial') & (col('startYear') <= 2030), True) \
                .otherwise(False)

filtered_df = filtered_df.withColumn("meet_conditions", conditions) \
                        .filter(col("meet_conditions") == True) \
                        .drop("meet_conditions")

filtered_df.groupBy("titleType").agg(min("startYear").alias("min_startYear"),
                                   max("startYear").alias("max_startYear"),
                                  ).show()

[Stage 47:>                                                         (0 + 1) / 1]

+------------+-------------+-------------+
|   titleType|min_startYear|max_startYear|
+------------+-------------+-------------+
|    tvSeries|         1927|         2029|
|tvMiniSeries|         1928|         2030|
|     tvMovie|         1928|         2025|
|   tvEpisode|         1928|         2030|
|       movie|         1894|         2025|
|   tvSpecial|         1931|         2026|
|       video|         1923|         2025|
|   videoGame|         1958|         2025|
|     tvShort|         1929|         2025|
|       short|         1874|         2025|
+------------+-------------+-------------+



In [18]:
conditions = when((col('titleType') == 'tvSeries') & (col('runtimeMinutes') <= 1500), True) \
              .when((col('titleType') == 'tvMiniSeries') & (col('runtimeMinutes') <= 1500), True) \
              .when((col('titleType') == 'tvMovie') & (col('runtimeMinutes') <= 500), True) \
              .when((col('titleType') == 'tvEpisode') & (col('runtimeMinutes') <= 300), True) \
              .when((col('titleType') == 'movie') & (col('runtimeMinutes') <= 500), True) \
              .when((col('titleType') == 'tvSpecial') & (col('runtimeMinutes') <= 1500), True) \
              .when((col('titleType') == 'video') & (col('runtimeMinutes') <= 1500), True) \
              .when((col('titleType') == 'videoGame') & (col('runtimeMinutes') <= 1500), True) \
              .when((col('titleType') == 'tvShort') & (col('runtimeMinutes') <= 200), True) \
              .when((col('titleType') == 'short') & (col('runtimeMinutes') <= 60), True) \

filtered_df = filtered_df.withColumn("meet_conditions", conditions) \
                     .filter(col("meet_conditions") == True) \
                     .drop("meet_conditions")

filtered_df.groupBy("titleType").agg(min("runtimeMinutes").alias("min_runtime"),
                                   max("runtimeMinutes").alias("max_runtime"),
                                   avg("runtimeMinutes").alias("avg_runtime")
                                  ).show()

[Stage 50:>                                                         (0 + 1) / 1]

+------------+-----------+-----------+------------------+
|   titleType|min_runtime|max_runtime|       avg_runtime|
+------------+-----------+-----------+------------------+
|    tvSeries|          1|       1500| 44.93794798080845|
|tvMiniSeries|          1|       1080| 73.10273502597785|
|     tvMovie|          1|        498| 71.46928069969016|
|   tvEpisode|          1|        300| 38.02999565004622|
|       movie|          1|        500| 88.90425103069171|
|   tvSpecial|          1|       1500| 83.93787915554478|
|       video|          1|       1445| 69.08648664366301|
|   videoGame|          1|       1495|100.70138888888889|
|     tvShort|          1|        120|12.146563047285465|
|       short|          0|         60|13.054938441485389|
+------------+-----------+-----------+------------------+



In [19]:
filtered_df.groupBy("titleType").agg(min("runtimeMinutes").alias("min_runtime"),
                                   max("runtimeMinutes").alias("max_runtime"),
                                   avg("runtimeMinutes").alias("avg_runtime")
                                  ).show()

filtered_df.groupBy("titleType").agg(min("startYear").alias("min_startYear"),
                                   max("startYear").alias("max_startYear"),
                                  ).show()

+------------+-----------+-----------+------------------+
|   titleType|min_runtime|max_runtime|       avg_runtime|
+------------+-----------+-----------+------------------+
|    tvSeries|          1|       1500| 44.93794798080845|
|tvMiniSeries|          1|       1080| 73.10273502597785|
|     tvMovie|          1|        498| 71.46928069969016|
|   tvEpisode|          1|        300| 38.02999565004622|
|       movie|          1|        500| 88.90425103069171|
|   tvSpecial|          1|       1500| 83.93787915554478|
|       video|          1|       1445| 69.08648664366301|
|   videoGame|          1|       1495|100.70138888888889|
|     tvShort|          1|        120|12.146563047285465|
|       short|          0|         60|13.054938441485389|
+------------+-----------+-----------+------------------+



[Stage 56:>                                                         (0 + 1) / 1]

+------------+-------------+-------------+
|   titleType|min_startYear|max_startYear|
+------------+-------------+-------------+
|    tvSeries|         1930|         2026|
|tvMiniSeries|         1936|         2026|
|     tvMovie|         1928|         2025|
|   tvEpisode|         1931|         2027|
|       movie|         1894|         2025|
|   tvSpecial|         1931|         2025|
|       video|         1930|         2025|
|   videoGame|         1975|         2025|
|     tvShort|         1936|         2025|
|       short|         1874|         2025|
+------------+-------------+-------------+



In [20]:
print("Before Filtering:")
print(f"Total number of records: {movies_df.count()}")
movies_df.select("titleType").groupby("titleType").count().show()

print("After Filtering:")
print(f"Total number of records: {filtered_df.count()}")
filtered_df.select("titleType").groupby("titleType").count().show()

Before Filtering:


Total number of records: 11442166


+------------+-------+
|   titleType|  count|
+------------+-------+
|    tvSeries| 276650|
|tvMiniSeries|  59556|
|     tvMovie| 149728|
|     tvPilot|      1|
|   tvEpisode|8797679|
|       movie| 705880|
|   tvSpecial|  51332|
|       video| 305529|
|   videoGame|  41927|
|     tvShort|  10541|
|       short|1043343|
+------------+-------+

After Filtering:


Total number of records: 3403012


[Stage 68:>                                                         (0 + 1) / 1]

+------------+-------+
|   titleType|  count|
+------------+-------+
|    tvSeries| 102962|
|tvMiniSeries|  20402|
|     tvMovie| 101988|
|   tvEpisode|1839100|
|       movie| 436600|
|   tvSpecial|  24726|
|       video| 206344|
|   videoGame|    432|
|     tvShort|   9136|
|       short| 661322|
+------------+-------+



In [21]:
movies_df.groupBy("titleType").agg(
    max("startYear").alias("max_startYear")
).orderBy("max_startYear", ascending=False).show()

[Stage 71:>                                                         (0 + 1) / 1]

+------------+-------------+
|   titleType|max_startYear|
+------------+-------------+
|       movie|         2031|
|tvMiniSeries|         2030|
|   tvEpisode|         2030|
|       short|         2030|
|    tvSeries|         2029|
|     tvMovie|         2026|
|   tvSpecial|         2026|
|       video|         2026|
|   videoGame|         2026|
|     tvShort|         2025|
|     tvPilot|         1991|
+------------+-------------+



In [22]:
movies_df.where(col("titleType") == 'tvPilot').show()

[Stage 74:>                                                         (0 + 1) / 1]

+----------+---------+------------+-------------+-------+---------+-------+--------------+------+
|    tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+----------+---------+------------+-------------+-------+---------+-------+--------------+------+
|tt15258334|  tvPilot|    TV Pilot|     TV Pilot|  false|     1991|   NULL|          NULL|    \N|
+----------+---------+------------+-------------+-------+---------+-------+--------------+------+



In [ ]:
This notebook leverages PySpark to efficiently explore and analyze the IMDb dataset, which contains structured information about movies, TV shows, ratings, crew, and industry professionals. Using PySpark’s distributed computing capabilities, we process large-scale data seamlessly, gaining insights into trends, ratings, genres, and more.